In [53]:
from docx import Document
from geojson import Polygon, MultiPolygon, Feature, FeatureCollection, dump
from pyproj import Proj
import utm

In [54]:
doc = Document('data/coords.docx')

In [55]:
table = doc.tables[0]

data = []

keys = None
for i, row in enumerate(table.rows):
    text = (cell.text for cell in row.cells)

    if i == 0:
        keys = tuple(text)
        continue

    row_data = dict(zip(keys, text))
    data.append(row_data)

In [56]:
myProj = Proj("+proj=utm +zone=36L, +south +ellps=WGS84")

In [57]:
def parseCoord(s):
    s = s.replace("\n", "")
    s = s.replace(":", "")
    s = s.replace(" ", "")
    s = s.replace(".", "")
    s = s.replace("36L", "")

    east, south = s.split("UTM")
    east = east.lstrip()

    coord = utm.to_latlon(float(east), float(south), 36, "L")
    
    return [coord[1], coord[0]]


In [58]:
features = []

for f in data:
    coords = []
    fc = f.copy()
    for i in range(1, 10):
        index = f'COORD{i}'
        coord_string = fc[index]
        fc.pop(index)
        if len(coord_string) > 4:
            coords.append(parseCoord(coord_string))

    if len(coords) > 0:
        coords.append(coords[0])

    polygon = Polygon([coords])
    features.append(Feature(geometry=polygon, properties=fc))

# add more features...
# features.append(...)

feature_collection = FeatureCollection(features)

with open('output/coords.geojson', 'w') as f:
   dump(feature_collection, f)